In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00


1. Data Downloading

In [ ]:
# download data from huggingface
from datasets import load_dataset
docs = load_dataset('cnn_dailymail', '1.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
docs.save_to_disk('/content/drive/My Drive/NLP/docs')

Saving the dataset (0/3 shards):   0%|          | 0/287113 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13368 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11490 [00:00<?, ? examples/s]

*2*. Data tokenization

In [ ]:
from transformers import AutoTokenizer
# Get the tokenizer - making words into numbers
tokenizer = AutoTokenizer.from_pretrained('t5-small')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
# prefix is needed for T5
prefix = 'summarize:'

def tokenization(docs, max_article_length=1024, max_summary_length=64):

    articles = [prefix + doc for doc in docs['article']]

    tokenized_articles = tokenizer(articles, max_length=max_article_length, truncation=True)

    tokenized_summaries = tokenizer(docs['highlights'], max_length=max_summary_length, truncation=True)

    tokenized_articles['labels'] = tokenized_summaries['input_ids']

    return tokenized_articles

In [ ]:
# tokenize the data
tokenized_docs = docs.map(tokenization, batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
tokenized_docs.save_to_disk('/content/drive/My Drive/NLP/tokenized_docs')

Saving the dataset (0/3 shards):   0%|          | 0/287113 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13368 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# keep only the tokens
tokens_docs = tokenized_docs.remove_columns(['article','highlights', 'id'])

In [ ]:
tokens_docs

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

3.Data Padding and spliting

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM
# get the model
model = TFAutoModelForSeq2SeqLM.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
from transformers import DataCollatorForSeq2Seq
# get a collator for data padding
collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
# process data
train_data = model.prepare_tf_dataset(
    tokens_docs['train'],
    shuffle=True,
    batch_size=8,
    tokenizer= tokenizer,
    collate_fn=collator,
)

val_data = model.prepare_tf_dataset(
    tokens_docs['validation'],
    shuffle=False,
    batch_size=8,
    tokenizer= tokenizer,
    collate_fn=collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


4. Training

In [ ]:
# Use Adam as the optimizer
import keras
optimizer = keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_data, validation_data=val_data, epochs=2, verbose=True)


Epoch 1/2
35889/35889 [==============================] - 10181s 283ms/step - loss: 1.8782 - val_loss: 1.6667
Epoch 2/2
35889/35889 [==============================] - 10111s 282ms/step - loss: 1.8230 - val_loss: 1.6437


In [ ]:
# save the model
model.save_pretrained("/content/drive/My Drive/NLP/tuned_T5")

5. Inference

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM
from google.colab import drive
drive.mount('/content/drive')
model_path = "/content/drive/My Drive/NLP/tuned_T5"

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer
# Get the tokenizer - making words into numbers
tokenizer = AutoTokenizer.from_pretrained('t5-small')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
from transformers import TFT5ForConditionalGeneration
model= TFT5ForConditionalGeneration.from_pretrained(model_path)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/drive/My Drive/NLP/tuned_T5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
from datasets import load_from_disk
dataset_path = "/content/drive/My Drive/NLP/docs/test"
test_dataset = load_from_disk(dataset_path)

In [ ]:
test_dataset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 11490
})

In [ ]:
test_inputs = test_dataset['article']

In [ ]:
len(test_inputs)

11490

In [ ]:
test_inputs_ids = [tokenizer('summarize:' + input, max_length=1024, return_tensors="tf", padding=True, truncation=True).input_ids for input in test_inputs]

In [ ]:
generated_summaries = []

for inputs_id in test_inputs_ids[:100]:
  outputs = model.generate(inputs_id, max_new_tokens=32, do_sample=False)
  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
  generated_summaries.append(summary)

In [ ]:
for inputs_id in test_inputs_ids[100:200]:
  outputs = model.generate(inputs_id, max_new_tokens=32, do_sample=False)
  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
  generated_summaries.append(summary)

In [ ]:
for inputs_id in test_inputs_ids[200:300]:
  outputs = model.generate(inputs_id, max_new_tokens=32, do_sample=False)
  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
  generated_summaries.append(summary)

In [ ]:
for inputs_id in test_inputs_ids[300:400]:
  outputs = model.generate(inputs_id, max_new_tokens=32, do_sample=False)
  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
  generated_summaries.append(summary)

In [ ]:
for inputs_id in test_inputs_ids[400:500]:
  outputs = model.generate(inputs_id, max_new_tokens=32, do_sample=False)
  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
  generated_summaries.append(summary)

In [ ]:
for inputs_id in test_inputs_ids[500:600]:
  outputs = model.generate(inputs_id, max_new_tokens=32, do_sample=False)
  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
  generated_summaries.append(summary)

In [ ]:
for inputs_id in test_inputs_ids[600:700]:
  outputs = model.generate(inputs_id, max_new_tokens=32, do_sample=False)
  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
  generated_summaries.append(summary)

In [ ]:
for inputs_id in test_inputs_ids[700:800]:
  outputs = model.generate(inputs_id, max_new_tokens=32, do_sample=False)
  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
  generated_summaries.append(summary)

In [ ]:
for inputs_id in test_inputs_ids[800:1000]:
  outputs = model.generate(inputs_id, max_new_tokens=32, do_sample=False)
  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
  generated_summaries.append(summary)

In [ ]:
len(generated_summaries)

1000

In [ ]:
import json
generated_summaries_json = json.dumps(generated_summaries)
with open("/content/drive/My Drive/NLP/generated_summaries.json", "w") as outfile:
    outfile.write(generated_summaries_json)

In [ ]:
ground_truth = test_dataset['highlights']

In [ ]:
import json
ground_truth_json = json.dumps(ground_truth)
with open("/content/drive/My Drive/NLP/ground_truth.json", "w") as outfile:
    outfile.write(ground_truth_json)

In [ ]:
len(ground_truth)

11490

references:

https://huggingface.co/docs/transformers/tasks/summarization

https://www.kaggle.com/code/dnayan/summarization-using-t5-huggingface

https://colab.research.google.com/github/huggingface/notebooks/blob/main/transformers_doc/en/summarization.ipynb